In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
#train function and stock the values in the lists
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # create lists to store the results for each epoch of the trained model
    train_loss = []
    val_loss = []
    train_acc = []
    val_acc = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                val_loss.append(epoch_loss)
                val_acc.append(epoch_acc)
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return [model,val_acc]

In [4]:
def grid_search_cv(param_grid, criterion, dataloaders, dataset_sizes):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    

    best_model = None
    best_acc = 0.0
    
    for lr in param_grid['lr']:
        for weight_decay in param_grid['weight_decay']:
            
            print(f'testing the following combination {lr} learning rate and {weight_decay} weight decay')
            
            model = models.resnet18(pretrained=True)
            model = model.to(device)
            
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
        
            trained_model = train_model(model, criterion, optimizer, scheduler)
            val_acc = trained_model[1]
            
            if best_acc < max(val_acc):
                best_acc = max(val_acc)
                best_model = copy.deepcopy(trained_model[0])
                best_lr = lr
                best_weight_decay = weight_decay
                
    print(f'the best parameters combination is a learning rate of {best_lr} and weight decay of {best_weight_decay}')
    return best_model, best_lr, best_weight_decay

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.empty_cache()

In [7]:
param_grid = {'lr': [0.001, 0.01, 0.1], 'weight_decay': [1e-4, 1e-6, 1e-8]}
criterion = nn.CrossEntropyLoss().to(device)

In [8]:
best_model = grid_search_cv(param_grid, criterion, dataloaders, dataset_sizes)

testing the following combination 0.001 learning rate and 0.0001 weight decay
Epoch 0/9
----------


C:\Users\thoma\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 0.9132 Acc: 0.7139
val Loss: 0.2191 Acc: 0.9279

Epoch 1/9
----------
train Loss: 0.4611 Acc: 0.8263
val Loss: 0.1695 Acc: 0.9471

Epoch 2/9
----------
train Loss: 0.3924 Acc: 0.8524
val Loss: 0.1440 Acc: 0.9532

Epoch 3/9
----------
train Loss: 0.3668 Acc: 0.8591
val Loss: 0.1365 Acc: 0.9564

Epoch 4/9
----------
train Loss: 0.3351 Acc: 0.8726
val Loss: 0.1119 Acc: 0.9654

Epoch 5/9
----------
train Loss: 0.3190 Acc: 0.8818
val Loss: 0.1030 Acc: 0.9675

Epoch 6/9
----------
train Loss: 0.3032 Acc: 0.8870
val Loss: 0.1103 Acc: 0.9650

Epoch 7/9
----------
train Loss: 0.2838 Acc: 0.8926
val Loss: 0.0996 Acc: 0.9693

Epoch 8/9
----------
train Loss: 0.2802 Acc: 0.8920
val Loss: 0.0970 Acc: 0.9693

Epoch 9/9
----------
train Loss: 0.2849 Acc: 0.8917
val Loss: 0.0969 Acc: 0.9689

Training complete in 12m 40s
Best val Acc: 0.969286
testing the following combination 0.001 learning rate and 1e-06 weight decay
Epoch 0/9
----------
train Loss: 0.9101 Acc: 0.7228
val Loss: 0.2075 Acc

train Loss: 0.2487 Acc: 0.9045
val Loss: 0.0968 Acc: 0.9657

Epoch 9/9
----------
train Loss: 0.2438 Acc: 0.9048
val Loss: 0.0882 Acc: 0.9721

Training complete in 11m 48s
Best val Acc: 0.972143
the best parameters combination is a learning rate of 0.01 and weight decay of 1e-06


In [14]:
#save the model
torch.save(best_model[0].state_dict(), 'best_ResNet.pth')